In [1]:
# %pip install selenium 

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time

In [3]:
def ScrapIndex(url,fromDate,toDate,index):
    driver = webdriver.Chrome()
    time.sleep(5)
    driver.get(url)
    time.sleep(5)
    WebDriverWait(driver, 10).until(
         EC.presence_of_element_located((By.ID, 'frm_datewise'))
    )
        
    dropdown = driver.find_element(By.CSS_SELECTOR, ".select2-selection--single")
    dropdown.click()
    options_container = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "select2-results"))
    )
    desired_option = driver.find_element(By.XPATH, f"//li[contains(text(), '{index}')]")
    desired_option.click()
    
    start_date = driver.find_element(By.ID, "fromDate")
    end_date = driver.find_element(By.ID, "toDate")
    start_date.clear()  # Clear the previous date
    end_date.clear()    # Clear the previous date
    start_date.send_keys(fromDate)
    end_date.send_keys(toDate)
    time.sleep(10)
    # Click search or submit button
    submit_button = driver.find_element(By.ID, "btn_indxhis_submit")
    submit_button.click()
    time.sleep(10)
   
    table = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, "myTable")))
    # table = driver.find_element(By.ID, "myTable")
    print (table)
    
    rows_per_page_select = driver.find_element(By.NAME, "myTable_length")
    select = Select(rows_per_page_select)
    select.select_by_value("50")
    time.sleep(2)
    data = []
    
    while True:
        rows = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.XPATH, ".//tr"))
        )
        
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//td")
            data.append([cell.text for cell in cells])

        try:
            next_button = driver.find_element(By.ID, "myTable_next") 
            if "disabled" in next_button.get_attribute("class"):
                break
            else:
                next_button.click()
                time.sleep(2)  # Wait for the next page to load
        except:
            break  
        data = [row for row in data if any(cell.strip() for cell in row)]
    
    print(data)
    df = pd.DataFrame(data, columns=["S.N","Open", "High", "Low", "Close","Change","%Change" ,"Turnover", "Date"])  # Modify columns as per table headers
    df.to_csv(f"{index}_history.csv", index=False)
    driver.quit()
                

    


In [4]:
url = "https://www.sharesansar.com/index-history-data"
fromDate = "2026-01-01"
toDate = "2026-01-25"
indeces = ["NEPSE Index","Banking SubIndex","HydroPower Index",]

for index in indeces :
    ScrapIndex(url,fromDate,toDate,index)
    time.sleep(3)

# Wait for the necessary elements to load



<selenium.webdriver.remote.webelement.WebElement (session="6cb6f5f5f1fde81c1eaff324a0cb5594", element="f.9F8CCA4B7AAB6F1FB414639A9A40A898.d.826A0E9913B87B975A08EE4C0DB2665E.e.101")>
[[], ['1', '2,716.25', '2,772.22', '2,715.78', '2,772.17', '57.55', '2.12', '13,785,729,645.05', '2026-01-25'], ['2', '2,706.36', '2,714.67', '2,693.04', '2,714.61', '9.23', '0.34', '9,073,978,098.78', '2026-01-22'], ['3', '2,717.70', '2,728.05', '2,695.10', '2,705.38', '-9.43', '-0.34', '8,923,650,346.99', '2026-01-21'], ['4', '2,679.10', '2,726.74', '2,678.81', '2,714.81', '42.26', '1.58', '11,858,421,070.67', '2026-01-20'], ['5', '2,640.54', '2,672.73', '2,640.13', '2,672.55', '31.12', '1.17', '8,402,797,775.59', '2026-01-18'], ['6', '2,636.51', '2,648.98', '2,633.00', '2,641.43', '1.52', '0.05', '6,209,438,906.36', '2026-01-14'], ['7', '2,633.21', '2,653.66', '2,631.78', '2,639.91', '4.90', '0.18', '6,990,484,039.45', '2026-01-13'], ['8', '2,643.18', '2,651.76', '2,628.98', '2,635.00', '-5.54', '-0.20',